# Annex GG Unified Demo

This notebook mirrors the Annex GG workflow using realistic synthetic adult-chest data (titanium rod; adjacent lesion).

## 1. Imports & Config

In [ ]:
import yaml, os
import numpy as np
import matplotlib.pyplot as plt

from mareval import (
    generate_synthetic_study, extract_roi, batch_extract_rois,
    build_pca_channels, apply_channels, cho_template, cho_decision_values,
    compute_auc_ci, paired_ttest_one_tailed, delta_auc_bias_assessment,
    make_parameter_grid, grid_to_index,
    plot_auc_heatmap, plot_delta_auc_heatmap,
)
cfg_path = os.path.join('configs','adult_chest_spinal_rod.yaml')
with open(cfg_path,'r') as f:
    cfg = yaml.safe_load(f)
cfg


## 2. Generate Synthetic Study

In [ ]:
study = generate_synthetic_study(cfg)
dose_levels = study['dose_levels']
contrast_levels = study['contrast_levels']
images = study['images']
roi_centers = study['roi_centers']
print('N dose:', len(dose_levels), 'N contrast:', len(contrast_levels))
print('Example key count:', len(images))


## 3. Preview grid counts and one example slice

In [ ]:
from mareval import make_parameter_grid
grid = make_parameter_grid(dose_levels, contrast_levels, cfg['realizations'], tuple(cfg['recon_types']), tuple(cfg['classes']))
print('Grid length:', len(grid))
sample_key = (0, 0, 0, 'FBP', 'present')
plt.figure()
plt.imshow(images[sample_key])
plt.title(f'Sample image key={sample_key}')
plt.axis('off')


## 4. ROI extraction on a couple of slices

In [ ]:
roi_size = tuple(cfg['roi_size'])
im_keys = [
    (0, 0, 0, 'FBP', 'present'),
    (10, 6, 0, 'MAR', 'absent'),
]
fig, axes = plt.subplots(1, len(im_keys), figsize=(8,4))
for ax, k in zip(axes, im_keys):
    ax.imshow(images[k])
    cy, cx = roi_centers[0]
    h, w = roi_size
    rect = plt.Rectangle((cx-w//2, cy-h//2), w, h, fill=False, linewidth=1.5)
    ax.add_patch(rect)
    ax.set_title(str(k))
    ax.axis('off')
plt.show()

# Extract a tiny training set for channels (mix of conditions)
train_imgs = [images[(0, 0, r, 'FBP', 'absent')] for r in range(3)] + [images[(10, 6, r, 'MAR', 'present')] for r in range(3)]
train_rois = batch_extract_rois(train_imgs, roi_centers, roi_size)
train_rois.shape


## 5. Build channels and CHO template

In [ ]:
U = build_pca_channels(train_rois, n_channels=int(cfg['channels']), whiten=True, random_state=0)

# Build class-wise channelized data for one cell to preview
def build_cell_data(d, j, recon):
    pos = []
    neg = []
    for r in range(cfg['realizations']):
        pos.append(extract_roi(images[(d, j, r, recon, 'present')], roi_centers[0], roi_size).ravel())
        neg.append(extract_roi(images[(d, j, r, recon, 'absent')], roi_centers[0], roi_size).ravel())
    pos = np.asarray(pos); neg = np.asarray(neg)
    ch_pos = (pos - pos.mean(0)) @ U
    ch_neg = (neg - neg.mean(0)) @ U
    return ch_pos, ch_neg

ch_pos, ch_neg = build_cell_data(5, 3, 'FBP')
w = cho_template(ch_pos, ch_neg, lambda_reg=float(cfg['lambda_reg']))
w.shape


## 6. Compute AUC per (dose, contrast) cell for MAR and FBP

In [ ]:
nD = len(dose_levels); nC = len(contrast_levels)
auc_fbp = np.zeros((nD, nC), dtype=float)
auc_mar = np.zeros((nD, nC), dtype=float)

for i, d in enumerate(dose_levels):
    for j, _ in enumerate(contrast_levels):
        # class data for FBP
        pos = []; neg = []
        for r in range(cfg['realizations']):
            pos.append(extract_roi(images[(d, j, r, 'FBP', 'present')], roi_centers[0], roi_size).ravel())
            neg.append(extract_roi(images[(d, j, r, 'FBP', 'absent')], roi_centers[0], roi_size).ravel())
        pos = np.asarray(pos); neg = np.asarray(neg)
        ch_pos = (pos - pos.mean(0)) @ U
        ch_neg = (neg - neg.mean(0)) @ U
        w = cho_template(ch_pos, ch_neg, lambda_reg=float(cfg['lambda_reg']))
        s_fbp = np.concatenate([ch_pos @ w, ch_neg @ w])
        y = np.array([1]*len(ch_pos) + [0]*len(ch_neg))
        res = compute_auc_ci(s_fbp, y, n_bootstrap=1000, random_state=123)
        auc_fbp[i, j] = res['auc']

        # MAR
        pos = []; neg = []
        for r in range(cfg['realizations']):
            pos.append(extract_roi(images[(d, j, r, 'MAR', 'present')], roi_centers[0], roi_size).ravel())
            neg.append(extract_roi(images[(d, j, r, 'MAR', 'absent')], roi_centers[0], roi_size).ravel())
        pos = np.asarray(pos); neg = np.asarray(neg)
        ch_pos = (pos - pos.mean(0)) @ U
        ch_neg = (neg - neg.mean(0)) @ U
        w = cho_template(ch_pos, ch_neg, lambda_reg=float(cfg['lambda_reg']))
        s_mar = np.concatenate([ch_pos @ w, ch_neg @ w])
        y = np.array([1]*len(ch_pos) + [0]*len(ch_neg))
        res = compute_auc_ci(s_mar, y, n_bootstrap=1000, random_state=321)
        auc_mar[i, j] = res['auc']

auc_fbp, auc_mar


## 7. Paired one-tailed t-tests & ΔAUC bias

In [ ]:
delta_auc = auc_mar - auc_fbp
# For a strict paired test, you would use replicate-level decision values;
# here we approximate by using per-cell AUCs as paired observations across replicates.
# A more complete pipeline could assemble replicate-level ΔAUC arrays.
delta_mean, p_one = paired_ttest_one_tailed(auc_fbp.ravel(), auc_mar.ravel())
bias = dict(delta_auc=float(delta_mean), p_value=float(p_one))
delta_mean, p_one, bias


## 8. Visualize AUC and ΔAUC heatmaps

In [ ]:
fig, ax = plot_auc_heatmap(auc_fbp, dose_levels, contrast_levels, title='FBP AUC')
fig, ax = plot_auc_heatmap(auc_mar, dose_levels, contrast_levels, title='MAR AUC')
fig, ax = plot_delta_auc_heatmap(delta_auc, dose_levels, contrast_levels, title='ΔAUC (MAR - FBP)')
plt.show()


## 9. Export Annex-GG-style tables

In [ ]:
import os
from mareval import save_auc_table_csv, save_delta_auc_table_csv
out_dir = 'outputs'
os.makedirs(out_dir, exist_ok=True)
save_auc_table_csv(os.path.join(out_dir,'auc_table.csv'), dose_levels, contrast_levels, auc_mar, auc_fbp)
save_delta_auc_table_csv(os.path.join(out_dir,'delta_auc_table.csv'), dose_levels, contrast_levels, delta_auc)
print('Tables written to', out_dir)
